In [1]:
from os import path

import astropy.coordinates as coord
from astropy.io import ascii
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astroquery.gaia import Gaia

login_file = path.expanduser('~/.gaia/archive.login')
if path.exists(login_file):
    with open(login_file, 'r') as f:
        username = f.readline().strip()
        password = f.readline().strip()
    print('Logging in with user: {0}'.format(username))
    Gaia.login(user=username, password=password)
    
    db_name = 'user_' + username
    
else:
    raise IOError('Login file not found')

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Logging in with user: apricewh
OK


In [6]:
ibata17 = ascii.read('../data/pal5/ibata2017.txt', format='cds')
ibata17['ibata_id'] = ['ibata{:02d}'.format(x) for x in range(len(ibata17))]

In [3]:
ibata17_ra = coord.Angle(['{}:{}:{}'.format(x['RAh'], x['RAm'], x['RAs']) for x in ibata17],
                         unit=u.hourangle)
ibata17_dec = coord.Angle(['{}{}:{}:{}'.format(x['DE-'], x['DEd'], x['DEm'], x['DEs']) for x in ibata17],
                         unit=u.degree)
ibata17_c = coord.SkyCoord(ibata17_ra, ibata17_dec)

In [7]:
new_tbl = at.Table()
new_tbl['ra_ibata'] = ibata17_ra.degree
new_tbl['dec_ibata'] = ibata17_dec.degree
new_tbl['vhelio'] = ibata17['HRV']
new_tbl['vhelio_err'] = ibata17['e_HRV']
new_tbl['ibata_id'] = ibata17['ibata_id']

In [5]:
tbl_loaded = False
for tbl in Gaia.load_tables(only_names=True):
    if tbl.name == '{}.{}'.format(db_name, 'ibata17_rgb'):
        tbl_loaded = True
        
if not tbl_loaded:
    Gaia.upload_table(upload_resource=new_tbl, 
                      table_name='ibata17_rgb')

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
Sending pytable.
Uploaded table 'ibata17_rgb'.


In [8]:
q = '''SELECT gaia.*, xm.ibata_id, xm.vhelio, xm.vhelio_err
FROM user_apricewh.ibata17_rgb as xm, gaiadr2.gaia_source as gaia
WHERE 1=CONTAINS(
  POINT('ICRS', xm.ra_ibata, xm.dec_ibata),
  CIRCLE('ICRS', gaia.ra, gaia.dec, 0.0006)
)
'''
job = Gaia.launch_job(q)
res = job.get_results()
res = res[res['parallax'] < 1.]

/Users/apricewhelan/anaconda/lib/python3.7/site-packages/astropy/table/column.py:981: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


In [10]:
np.unique(res['ibata_id']).size, len(res)

(153, 154)

In [31]:
x, counts = np.unique(res['ibata_id'], return_counts=True)

In [33]:
res[~np.isin(res['ibata_id'], x[counts > 1])].write('../data/pal5/ibata2017_with_gaia.csv')